# 1. Setup

In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

# 2. Load Data

In [ ]:
data_dir = '../data/raw'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('../data/raw', batch_size=16, image_size=(256, 256), color_mode='rgb', shuffle=True)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

# 3. Scale Data

In [ ]:
data = data.map(lambda x, y: (x / 255.0, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()
batch[0].max()

# 4. Split Data

In [ ]:
train_size = int(len(data)*0.75)
val_size = int(len(data)*0.17)
test_size = int(len(data)*0.08)

In [ ]:
train_data = data.take(train_size)
val_data = data.skip(train_size).take(val_size)
test_data = data.skip(train_size + val_size).take(test_size)

# 5. Build Convolutional Neural Network

In [ ]:
# Much smaller L2 regularization
reg_factor = 0.005

In [ ]:
model = Sequential([
    # First Convolutional Block - reduced initial filters
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3),
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(reg_factor)),
        BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Second Convolutional Block
    Conv2D(64, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Third Convolutional Block
    Conv2D(128, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Fourth Convolutional Block
    Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Flatten the output and add dense layers
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.4),  # Slightly reduced dropout for longer training
    Dense(128, activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(24, activation='softmax')
])

In [ ]:
optimizer = Adam(learning_rate=0.0001)

In [ ]:
model.compile(
        optimizer=optimizer,
        loss=tf.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

In [ ]:
model.summary()

# 7. Train model

In [ ]:
logdir = r'../data/logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Enhanced training configuration for longer training
training_config = {
    'epochs': 20,  # Increased epochs
    'batch_size': 16,
    'callbacks': [
        EarlyStopping(
            monitor='val_loss',
            patience=15,  # Increased patience
            restore_best_weights=True,
            min_delta=0.0001  # Smaller improvement threshold
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=8,  # Increased patience
            min_lr=1e-7,  # Lower minimum learning rate
            min_delta=0.0001
        ),
        tensorboard_callback
    ]
}

In [ ]:
# Data augmentation for better generalization during longer training
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

In [ ]:
augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x,training=True), y))

In [ ]:
history = model.fit(
    augmented_train_data,
    validation_data=val_data,
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=training_config['callbacks']
)

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate model

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test_data.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test model

In [ ]:
img = cv2.imread()
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if ( yhat >= 0) and ( yhat <= 1):
    print('Card is 7 blue')
else:
    print('Card is not 7 blue')